## Flüchtlingskrise Sentiment Analysis
### Emily Martin, eem80@pitt.edu

In [1]:
# Importing necessary libraries
import pandas as pd
import pickle
import nltk

## The data
#### Shape and acquisition
- Using the four scripts in my repo: [Süddeutsche_zeitung](https://github.com/Data-Science-for-Linguists-2021/Fluechtlingskrise-Sentiment-Analysis/blob/main/Süddeutsche_zeitung.ipynb), [taz](https://github.com/Data-Science-for-Linguists-2021/Fluechtlingskrise-Sentiment-Analysis/blob/main/taz.ipynb), [zeit](https://github.com/Data-Science-for-Linguists-2021/Fluechtlingskrise-Sentiment-Analysis/blob/main/zeit.ipynb) and [Junge Freiheit](https://github.com/Data-Science-for-Linguists-2021/Fluechtlingskrise-Sentiment-Analysis/blob/main/Junge%20Freiheit.ipynb) I was able to scrape the sites for news articles from 2015 using the search terms 'Flüchtling' (refugee) and/or 'Migranten' (migrants). 
- The actual number of articles varies widely per site because of ease of scraping and simply overall newspaper size. For Die TAZ there are 100 articles, from manually compiled links, for  Der Zeit there are 573, from links collected through their API, for Der Süddeutsche Zeitung there are 982 and for Junge Freiheit there are 60. 
- After collecting these articles in the scripts I made them into dataframes which I then pickled. However these pickled files are not available through my repo due to copywrite.

### A quick look at each source

#### Der Zeit
- Der Zeit is one of the largest weekly newspapers in Germany, it is centrist/liberal in its political leanings and kindly supports an API.

In [125]:
# Unpickle the dataframes
zeit_df = pd.read_pickle("zeit_df.pkl")
print(zeit_df.shape)
zeit_df.head()

(573, 9)


,title,href,text,release_date,word_count,sent_count,toks,types,TTR
0,Mahmood im Schilderwald,http://www.zeit.de/2015/51/fuehrerschein-fluec...,Als er vor über zehn Jahren Autofahren gelernt...,2015-12-31T02:51:37Z,1175,99,1390,608,0.437410
1,Zwei zähe Einzelgänger,http://www.zeit.de/2015/51/vorbereitung-auf-da...,"Wo Zou Lei herkommt, ist das Leben nicht leich...",2015-12-31T01:56:01Z,1155,75,1363,686,0.503302
2,Fortsetzung folgt – jetzt,http://www.zeit.de/2016/01/geschichten-2015-fo...,"Lok Leipzig ist ratlos, was aus Mario Basler w...",2015-12-30T09:00:08Z,313,26,372,229,0.615591
3,Anhaltend hohe Flüchtlingszahlen auf Balkanroute,http://www.zeit.de/gesellschaft/2015-12/slowen...,Auch zum Jahresende kommen weiter täglich Taus...,2015-12-29T22:14:02Z,362,23,402,236,0.587065
4,Laut Özoğuz schürt Union Vorurteile gegen Flüc...,http://www.zeit.de/politik/deutschland/2015-12...,Opposition und Koalitionspartner kritisieren d...,2015-12-29T08:24:55Z,379,24,447,237,0.530201


In [126]:
# A little about this dataframe:
zeit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         573 non-null    object 
 1   href          573 non-null    object 
 2   text          573 non-null    object 
 3   release_date  573 non-null    object 
 4   word_count    573 non-null    int64  
 5   sent_count    573 non-null    int64  
 6   toks          573 non-null    int64  
 7   types         573 non-null    int64  
 8   TTR           573 non-null    float64
dtypes: float64(1), int64(4), object(4)
memory usage: 40.4+ KB


In [127]:
zeit_df.describe()
# min of 1, there is at least one article where the link was broken

,word_count,sent_count,toks,types,TTR
count,573.000000,573.000000,573.000000,573.000000,573.000000
mean,567.130890,36.458988,657.005236,350.291449,0.555734
std,239.954035,16.763278,283.250269,122.933494,0.065913
min,1.000000,1.000000,1.000000,1.000000,0.360000
25%,393.000000,25.000000,447.000000,264.000000,0.513699
50%,562.000000,34.000000,654.000000,350.000000,0.552318
75%,701.000000,45.000000,812.000000,424.000000,0.587719
max,1945.000000,147.000000,2250.000000,880.000000,1.000000


### Die TAZ
-  Die TAZ (Die Tageszeitung) is a daily German newspaper with a modest circulation, it leans left-wing/green and is the most left-ist of the sources. 

In [128]:
# Unpickle and a quick look at the dataframe
taz_df = pd.read_pickle("taz_df.pkl")
print(taz_df.shape)
taz_df.head()

(100, 4)


,href,text,word_count,date
0,https://taz.de/Essay-Journalismus-und-Zuwander...,Deutschland hat sich verändert. Die Redaktione...,1232,2015-12-31
1,https://taz.de/Fluechtlingsdebatte-in-den-USA/...,Nach den Anschlägen von Paris wollen nur noch ...,786,2015-11-17
2,https://taz.de/Kommentar-Verfassungsschutz/!50...,Die Reform des V-Leute-Wesens ist eine Charmeo...,266,2015-03-25
3,https://taz.de/NPD-Invasion-in-Fluechtlingsunt...,NPD-Landtagsabgeordnete besuchten eine Erstauf...,561,2015-09-28
4,https://taz.de/Misshandlung-von-Fluechtlingen-...,Per Referendum will Premier Orbán rechtswidrig...,471,2015-04-28


In [6]:
taz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   href        100 non-null    object
 1   text        100 non-null    object
 2   word_count  100 non-null    int64 
 3   date        100 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.2+ KB


In [7]:
taz_df.describe()
# No broken links/problem areas

,word_count
count,100.000000
mean,799.430000
std,458.021332
min,209.000000
25%,495.000000
50%,674.000000
75%,948.000000
max,2846.000000


### Der Süddeutsche Zeitung
- Der Süddeustche Zeitung is a daily newspaper with a very wide ciruclation (second largest after Der Zeit), it leans left-liberal.

In [129]:
# Unpickle and aa quick look at the data
sz_df = pd.read_pickle("sz_df.pkl")
print(sz_df.shape)
sz_df.head()

(982, 8)


,href,text,date,word_count,sent_count,toks,types,TTR
0,https://www.sueddeutsche.de/politik/migration-...,Berlin (dpa) - Die Bundesländer haben für die ...,"27. Dezember 2015, 2:45 Uhr",89,5,103,83,0.805825
1,https://www.sueddeutsche.de/politik/migration-...,Rom (dpa) - Im Mittelmeer vor Italien sind auc...,"26. Dezember 2015, 20:51 Uhr",62,4,72,56,0.777778
2,https://www.sueddeutsche.de/kultur/rueckblick-...,1 / 12 Quelle: 20th Century Fox Südseefilme si...,"26. Dezember 2015, 17:57 Uhr",1818,87,2174,1004,0.461822
3,https://www.sueddeutsche.de/politik/rueckblick...,Bei dem Blick zurück auf das Jahr 2015 stechen...,"26. Dezember 2015, 16:00 Uhr",451,28,512,312,0.609375
4,https://www.sueddeutsche.de/politik/fluechtlin...,Nach einem Brandanschlag auf eine noch nicht f...,"26. Dezember 2015, 15:43 Uhr",387,26,451,274,0.607539


In [130]:
sz_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 982 entries, 0 to 999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   href        982 non-null    object 
 1   text        982 non-null    object 
 2   date        982 non-null    object 
 3   word_count  982 non-null    int64  
 4   sent_count  982 non-null    int64  
 5   toks        982 non-null    int64  
 6   types       982 non-null    int64  
 7   TTR         982 non-null    float64
dtypes: float64(1), int64(4), object(3)
memory usage: 69.0+ KB


In [131]:
sz_df.describe()
# Fairly short articles
# min of 1 suggests a faulty article made it through...

,word_count,sent_count,toks,types,TTR
count,982.000000,982.000000,982.000000,982.000000,982.000000
mean,375.175153,24.216904,438.845214,240.369654,0.618964
std,280.159402,19.202361,329.773222,146.551981,0.113869
min,4.000000,1.000000,5.000000,5.000000,0.307692
25%,129.500000,8.000000,155.500000,107.250000,0.532525
50%,342.000000,21.000000,397.500000,233.500000,0.591833
75%,536.000000,34.000000,636.000000,335.750000,0.693488
max,2402.000000,136.000000,2767.000000,1172.000000,1.000000


### Junge Freiheit
-  Junge Freiheit is a small weekly newspaper with fairly strong right-wing leanings

In [132]:
# Unpickle and a quick look at the data
jf_df = pd.read_pickle("jf_df.pkl")
print(jf_df.shape) # This is the smallest sample
jf_df.head()

(76, 8)


,href,text,date,word_count,sent_count,toks,types,TTR
0,https://jungefreiheit.de/kultur/gesellschaft/2...,Um ihn herum hängen Arbeiter gerade die Weihna...,19. November 2015,1348,77,1569,658,0.419375
1,https://jungefreiheit.de/politik/deutschland/2...,POTSDAM. Brandenburgs AfD-Chef Alexander Gaula...,18. November 2015,385,20,456,281,0.616228
2,https://jungefreiheit.de/debatte/kommentar/201...,Die Norwegerin Linda Hagen ist immer noch ganz...,05. November 2015,171,7,203,152,0.748768
3,https://jungefreiheit.de/politik/deutschland/2...,"ERFURT. Asylbewerber, die mit der Deutschen Ba...",04. November 2015,191,12,227,157,0.691630
4,https://jungefreiheit.de/politik/ausland/2015/...,TRIPOLIS. Der libysche „Allgemeine Volkskongre...,04. November 2015,262,17,307,181,0.589577


In [133]:
jf_df.info()
# All non-null, which is good. Can't really afford to lose more articles from this source

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 0 to 75
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   href        76 non-null     object 
 1   text        76 non-null     object 
 2   date        76 non-null     object 
 3   word_count  76 non-null     int64  
 4   sent_count  76 non-null     int64  
 5   toks        76 non-null     int64  
 6   types       76 non-null     int64  
 7   TTR         76 non-null     float64
dtypes: float64(1), int64(4), object(3)
memory usage: 5.3+ KB


In [134]:
jf_df.describe()

,word_count,sent_count,toks,types,TTR
count,76.000000,76.000000,76.000000,76.000000,76.000000
mean,469.921053,26.750000,550.736842,296.605263,0.586567
std,337.462759,18.807179,390.707269,172.371775,0.079762
min,126.000000,6.000000,150.000000,117.000000,0.419375
25%,194.500000,12.750000,229.750000,147.000000,0.531144
50%,343.500000,18.000000,408.500000,228.000000,0.585985
75%,747.500000,36.500000,860.500000,442.250000,0.645009
max,1348.000000,86.000000,1569.000000,725.000000,0.780000


## Sentiment Analysis

In [16]:
# Step 1
# Prepare data: remove stop words

# Lemmatization (seems better than stemming)?
# SpaCy has a german lemmetizer (and ger stopwords)


In [17]:
# More notes (3/31): Make a spacy pipeline with sentiments analysis as part of it? (how to use sentiws and how 
# to loop through a list of the texts from each array - figure it out!)

# Or: Use SpaCy for tokenization, lemmatization, removing stop words by creating custom functions? Look into both 
# options. 
# I definitely need to get something with labels, so sentiws seems good
# Much to do....

In [18]:
# Pipeline ideas: tokenization, remove stop words, lemmatization, sentiment analysis

In [19]:
import spacy


In [97]:
from spacy_sentiws import spaCySentiWS
from spacy.language import Language
from spacy.language import Language

nlp = spacy.load("de_core_news_sm", disable=['ner']) #Don't need entity recognition

In [98]:
# Have to set it up this way (prone to not working?) may have to trouble shoot
@Language.component("sentiws")
def my_component(doc):
    return doc

# Usage as a function
Language.component("my_component2", func=my_component)

sentiws = spaCySentiWS(sentiws_path='/Users/emilymartin/Documents/data/SentiWS_v2')

nlp.add_pipe('sentiws')
doc = nlp('Die Dummheit der Unterwerfung blüht in hübschen Farben.')

for token in doc:
    print('{}, {}, {}'.format(token.text, token._.sentiws, token.pos_))
    
print(nlp.pipeline)

Die, None, DET
Dummheit, -0.4877, NOUN
der, None, DET
Unterwerfung, -0.3279, NOUN
blüht, 0.2028, VERB
in, None, ADP
hübschen, 0.4629, ADJ
Farben, None, NOUN
., None, PUNCT
[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7ffea1df3110>), ('tagger', <spacy.pipeline.tagger.Tagger object at 0x7ffea3df6770>), ('morphologizer', <spacy.pipeline.morphologizer.Morphologizer object at 0x7ffea3df6110>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x7ffebf37e750>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x7ffea40ca370>), ('lemmatizer', <spacy.pipeline.lemmatizer.Lemmatizer object at 0x7ffe9f00adc0>), ('sentiws', <function my_component at 0x7ffea9344680>)]


In [38]:
from  spacy.lang.de.stop_words import STOP_WORDS
stop = STOP_WORDS


# It works!!!! 
foo = nlp(zeit_df.text[0])

foo_nostop = [token for token in foo if not token.is_stop]
    
print(foo_nostop[:50])

# Should I define a function to do this? Or find a way to write one into the pipeline?

[Autofahren, gelernt, ,, brauchte, Mahmood, 
      , Shaker, Führerschein, ., Wozu, ?, Heimat, Irak, 
      , Fahrerlaubnis, gefragt, ,, mal, Polizist, ,, angehalten, ., Verkehrsregeln, 
      , gebe, ,, halte, ., ", Nachbarn, lässt, Vorfahrt, ,, 
      , Fremden, ", ,, Shaker, ., Schilder, dienten, grobe, Orientierung, ,, Ampeln, 
      , fast, kaputt, ., 
 ]


In [77]:
# Trying out the sentiment analyzer on the first 5 texts from zeit
docs = zeit_df.text[:5]

senti_sc = []


for doc in nlp.pipe(docs):
    for token in doc:
        print('{}, {}'.format(token.text, token._.sentiws))
    
#Huh most are None, may have to rethink?
# Flüchtling (refugee) is negative, interesting!!

Als, None
er, None
vor, None
über, None
zehn, None
Jahren, None
Autofahren, None
gelernt, 0.2492
hat, None
,, None
brauchte, None
Mahmood, None

      , None
Shaker, None
keinen, None
Führerschein, None
., None
Wozu, None
auch, None
?, None
In, None
seiner, None
Heimat, None
Irak, None
hat, None
ihn, None
nie, None
jemand, None
nach, None
einer, None

      , None
Fahrerlaubnis, None
gefragt, None
,, None
nicht, None
mal, None
der, None
Polizist, None
,, None
der, None
ihn, None
einmal, None
angehalten, None
hat, None
., None
Verkehrsregeln, None

      , None
gebe, None
es, None
zwar, None
,, None
aber, None
kaum, None
jemand, None
halte, None
sich, None
daran, None
., None
", None
Dem, None
Nachbarn, None
lässt, None
man, None
die, None
Vorfahrt, None
,, None
einem, None

      , None
Fremden, None
nicht, None
", None
,, None
sagt, None
Shaker, None
., None
Schilder, None
dienten, None
nur, None
als, None
grobe, -0.3495
Orientierung, None
,, None
die, None
Ampeln, None
seien, None

 

,, None
den, None
man, None
erwischt, None
,, None
hat, None
automatisch, None
gegen, None
den, None
Patriot, None
Act, None
verstoßen, -0.3521
,, None
das, None
genügt, None
., None
In, None
der, None
Folge, None
schlägt, -0.0306
sie, None
sich, None
in, None
New, None
York, None
mit, None
Jobs, None
bei, None
chinesischen, None
Imbissen, None
durch, None
,, None
wo, None
sowieso, None
jeder, None
illegal, None
ist, None
,, None
und, None
sie, None
jedenfalls, None
in, None
keiner, None
größeren, 0.3694
Gefahr, -1.0
schwebt, None
als, None
alle, None
anderen, None
., None

 , None
Parallel, None
dazu, None
wird, None
die, None
Geschichte, None
von, None
Skinner, None
erzählt, None
(, None
dass, None
er, None
auch, None
einen, None
Vornamen, None
hat, None
,, None
Brad, None
,, None
spielt, None
so, None
gut, None
wie, None
keine, None
Rolle, None
), None
., None
Er, None
ist, None
gerade, None
von, None
seinem, None
dritten, None
Einsatz, None
im, None
Irak, None
zurückgekehrt, None
u

den, None
Knüppel, None
., None

 , None
Von, None
Martin, None
Machowecz, None

, None
Auch, None
zum, None
Jahresende, None
kommen, None
weiter, None
täglich, None
Tausende, None
Flüchtlinge, -0.0048
über, None
die, None
Balkanroute, None

 , None
nach, None
Österreich, None
und, None
Deutschland, None
., None
Allein, None
am, None
Dienstag, None
seien, None
bis, None
zum, None

, None
Mittag, None
in, None
Slowenien, None
2.000, None
Migranten, None
gezählt, None
worden, None
,, None
berichtete, None
der, None
slowenische, None

, None
Innenstaatssekretär, None
Boštjan, None
Šefic, None
., None
Am, None
Vortag, None
seien, None
es, None
4.200, None

 , None
gewesen, None
,, None
über, None
das, None
Weihnachtswochenende, None
seien, None
fast, None
11.000, None
Menschen, None

, None
registriert, None
worden, None
., None

 , None
Seit, None
Ungarn, None
Mitte, None
Oktober, None
seine, None
Grenzen, None
mit, None
einem, None

, None
Zaun, None
abgeriegelt, None
hat, None
und, None

In [ ]:
text = t
doc = nlp(text)

for token in doc:
    print(token, token.pos_, token.tag_)
    
for token in doc:
    print(token, token.morph)
    
for token in doc:
    print(token, token.lemma_)

In [ ]:
len(doc)
#type(doc)

In [33]:
with nlp.select_pipes(enable="lemmatizer"):
    doc = nlp("machen, mache, machst, macht, machte, gemacht")
for t in doc:
    print(t, t.lemma_)
    
print(nlp.pipeline)

machen machen
, ,
mache machen
, ,
machst machen
, ,
macht machen
, ,
machte machen
, ,
gemacht machen
[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7ffeb2e09470>), ('tagger', <spacy.pipeline.tagger.Tagger object at 0x7ffeb2e20c50>), ('morphologizer', <spacy.pipeline.morphologizer.Morphologizer object at 0x7ffeb2e20b90>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x7ffed613a910>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x7ffea48430a0>), ('lemmatizer', <spacy.pipeline.lemmatizer.Lemmatizer object at 0x7ffeddade6e0>), ('sentiws', <function my_component at 0x7ffebe8464d0>)]


In [102]:
# I need to set it this way, or I run into errors later
from spacy.tokens import Doc
Doc.set_extension("sentiws", default=True)

In [240]:
url_lst = zeit_df.href.tolist()
doc_lst = zeit_df.text.tolist()
scores = {}

# zip function = super cool and useful!
for url, doc in zip(url_lst, nlp.pipe(doc_lst)):
    x = [token._.sentiws for token in doc if token._.sentiws is not None]
    scores[url] = x

#scores
# hmmm I wonder what the 0.004 score is, smoothing?

In [188]:
import statistics
for value in scores.values():
    try:
        print(statistics.mean(value), statistics.median(value))
    except:
        print('issue')
        pass

-0.024352727272727272 0.004
-0.05217014925373135 -0.0048
-0.02824 -0.00039999999999999975
-0.04213846153846154 -0.0048
-0.06522380952380952 -0.0048
-0.08204375 0.004
-0.08442535211267606 -0.0048
-0.05430714285714286 -0.0048
0.02783333333333333 0.004
0.02210625 -0.0048
0.014642857142857145 0.004
-0.019418181818181817 -0.0048
-0.02047727272727273 0.004
-0.009474390243902439 0.004
-0.07574545454545455 -0.0048
-0.004653125000000003 0.004
-0.093375 -0.0048
-0.08793333333333334 -0.0048
-0.10252647058823529 -0.0048
-0.0639625 -0.0048
-0.0314 0.004
-0.07730344827586207 -0.0048
-0.09598571428571429 -0.0048
-0.05764901960784314 0.004
-0.08228571428571428 -0.0048
-0.059333333333333335 0.004
-0.04571290322580645 0.004
-0.148885 -0.0048
-0.010600000000000002 0.004
-0.11352857142857142 -0.0048
-0.039073913043478264 -0.0048
-0.21515600000000001 -0.0048
0.0021551724137931017 -0.0048
0.032724 0.004
-0.11754444444444444 -0.0048
0.00010000000000000085 0.004
-0.12325918367346939 -0.0048
0.0228277777777777

-0.1308590909090909 -0.0048
-0.12450338983050847 -0.0048
-0.11388787878787879 -0.0048
-0.06675555555555555 0.004
-0.16773214285714286 -0.0048
-0.13100454545454546 -0.0048
-0.062126027397260274 0.004
-0.049673684210526314 -0.0048
-0.14748571428571428 -0.0048
-0.10424444444444443 0.004
-0.020050000000000002 -0.00039999999999999975
-0.13132857142857143 -0.0048
-0.12184 -0.0565
0.0393448275862069 0.004
-0.04178305084745763 0.004
-0.032993939393939395 0.004
0.07995714285714285 0.004
-0.028279166666666668 0.004
-0.038055882352941175 0.004
-0.07780769230769231 -0.0048
0.058135 0.004
-0.020206666666666668 0.004
-0.060858333333333334 0.004
0.04622222222222222 0.004
-0.06617017543859649 0.004
-0.03165384615384615 0.004
-0.13513684210526317 -0.0048
-0.1524659090909091 -0.0048
-0.05681153846153846 -0.00039999999999999975
-0.18182413793103447 -0.2088
-0.009008163265306124 -0.0048
0.014316666666666669 0.004
issue
-0.07587142857142858 -0.0048
-0.2036558823529412 -0.18159999999999998
0.076384615384615

In [241]:
for k, v in scores.items():
    try:
        scores[k] = statistics.mean(v)
    except:
        scores[k] = 0.0
        pass
#scores

In [252]:
zeit_df['score']= zeit_df['href'].map(scores)
zeit_df.head()

,title,href,text,release_date,word_count,sent_count,toks,types,TTR,score
0,Mahmood im Schilderwald,http://www.zeit.de/2015/51/fuehrerschein-fluec...,Als er vor über zehn Jahren Autofahren gelernt...,2015-12-31T02:51:37Z,1175,99,1390,608,0.437410,-0.024353
1,Zwei zähe Einzelgänger,http://www.zeit.de/2015/51/vorbereitung-auf-da...,"Wo Zou Lei herkommt, ist das Leben nicht leich...",2015-12-31T01:56:01Z,1155,75,1363,686,0.503302,-0.052170
2,Fortsetzung folgt – jetzt,http://www.zeit.de/2016/01/geschichten-2015-fo...,"Lok Leipzig ist ratlos, was aus Mario Basler w...",2015-12-30T09:00:08Z,313,26,372,229,0.615591,-0.028240
3,Anhaltend hohe Flüchtlingszahlen auf Balkanroute,http://www.zeit.de/gesellschaft/2015-12/slowen...,Auch zum Jahresende kommen weiter täglich Taus...,2015-12-29T22:14:02Z,362,23,402,236,0.587065,-0.042138
4,Laut Özoğuz schürt Union Vorurteile gegen Flüc...,http://www.zeit.de/politik/deutschland/2015-12...,Opposition und Koalitionspartner kritisieren d...,2015-12-29T08:24:55Z,379,24,447,237,0.530201,-0.065224


In [245]:
def neg_pos(score):
    if score < 0.0:
        return 'neg'
    elif score > 0.0:
        return 'pos'
    else:
        return None

In [253]:
zeit_df.score = zeit_df['score'].map(lambda s: neg_pos(s))
zeit_df.head()

,title,href,text,release_date,word_count,sent_count,toks,types,TTR,score
0,Mahmood im Schilderwald,http://www.zeit.de/2015/51/fuehrerschein-fluec...,Als er vor über zehn Jahren Autofahren gelernt...,2015-12-31T02:51:37Z,1175,99,1390,608,0.437410,neg
1,Zwei zähe Einzelgänger,http://www.zeit.de/2015/51/vorbereitung-auf-da...,"Wo Zou Lei herkommt, ist das Leben nicht leich...",2015-12-31T01:56:01Z,1155,75,1363,686,0.503302,neg
2,Fortsetzung folgt – jetzt,http://www.zeit.de/2016/01/geschichten-2015-fo...,"Lok Leipzig ist ratlos, was aus Mario Basler w...",2015-12-30T09:00:08Z,313,26,372,229,0.615591,neg
3,Anhaltend hohe Flüchtlingszahlen auf Balkanroute,http://www.zeit.de/gesellschaft/2015-12/slowen...,Auch zum Jahresende kommen weiter täglich Taus...,2015-12-29T22:14:02Z,362,23,402,236,0.587065,neg
4,Laut Özoğuz schürt Union Vorurteile gegen Flüc...,http://www.zeit.de/politik/deutschland/2015-12...,Opposition und Koalitionspartner kritisieren d...,2015-12-29T08:24:55Z,379,24,447,237,0.530201,neg


In [251]:
t = zeit_df[zeit_df.score == 'pos'] 
t    

,title,href,text,release_date,word_count,sent_count,toks,types,TTR,score
8,Italienische Küstenwache rettet 751 Bootsflüch...,http://www.zeit.de/gesellschaft/2015-12/fluech...,Die italienische Küstenwache hat 751 \nBootsfl...,2015-12-25T20:23:32Z,139,9,159,113,0.710692,pos
9,180 Flüchtlinge stürmen auf spanische Exklave,http://www.zeit.de/gesellschaft/2015-12/fluech...,"Mehrere Hundert Flüchtlinge haben versucht, vo...",2015-12-25T14:03:30Z,205,12,229,150,0.655022,pos
10,Papst dankt Flüchtlingshelfern in aller Welt,http://www.zeit.de/gesellschaft/zeitgeschehen/...,Vor geschätzt 40.000 Menschen hat Papst Franzi...,2015-12-25T13:43:49Z,438,18,510,282,0.552941,pos
32,Türkei-Abkommen senkt Flüchtlingszahl kaum,http://www.zeit.de/politik/2015-12/tuerkei-gri...,"Die Zahl der Flüchtlinge, die aus der Türkei i...",2015-12-17T02:35:49Z,352,24,396,228,0.575758,pos
33,Wie ein Provinzmuseum aus den Achtzigern,http://www.zeit.de/sport/2015-12/fussball-muse...,Das Deutsche Fußballmuseum in Dortmund hat sic...,2015-12-16T12:48:20Z,656,45,750,397,0.529333,pos
...,...,...,...,...,...,...,...,...,...,...
534,Dresdner Zustände,http://www.zeit.de/politik/deutschland/2015-01...,Am kommenden Montag wird es keine Pegida-Demo ...,2015-01-28T18:23:20Z,414,32,496,283,0.570565,pos
548,"Fremd in Paris, Wiesbaden, Rostock-Lichtenhagen",http://www.zeit.de/kultur/film/2015-01/samba-w...,"Eingängige\nRhythmen, Tänzerinnen in Zwanziger...",2015-01-21T14:53:11Z,742,35,877,473,0.539339,pos
556,"""Wir sind die einzigen, die an Frankreich glau...",http://www.zeit.de/politik/ausland/2015-01/mar...,Eine Fotomontage macht\ngerade im Internet die...,2015-01-14T13:42:57Z,555,35,656,364,0.554878,pos
565,Der Marsch der Republik,http://www.zeit.de/politik/ausland/2015-01/par...,Das Bild ist durchaus eines historischen Tages...,2015-01-11T20:07:20Z,710,37,812,455,0.560345,pos
